In [1]:
import torch 
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from torchvision import transforms
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import time
from torchvision.transforms import v2
import random

In [2]:
transform = transforms.Compose([transforms.ToTensor()])

train_set = ImageFolder(root='cinic10/versions/1/train//', transform=transform)
validate_set = ImageFolder(root='cinic10/versions/1/valid//', transform=transform)
test_set = ImageFolder(root='cinic10/versions/1/test//', transform=transform)

class_names = train_set.classes
num_labels = len(class_names)

In [3]:
from torchvision.models import efficientnet_b0
from torchvision.models import resnext50_32x4d 

In [4]:
def prepare_model_efficient_net(lr):
    model = efficientnet_b0(pretrained=False)
    model.classifier[1] = nn.Linear(in_features=1280, out_features=num_labels, bias=True)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
    criterion = torch.nn.CrossEntropyLoss()

    return model, optimizer, criterion

In [6]:
transform_default = transforms.Compose([transforms.ToTensor()])

def prepare_transforms():
    
    train_set = ImageFolder(root='cinic10/versions/1/train//', transform=transform_default)
    validate_set = ImageFolder(root='cinic10/versions/1/valid//', transform=transform_default)
    test_set = ImageFolder(root='cinic10/versions/1/test//', transform=transform_default)
        
    data_loader = DataLoader(train_set, batch_size=128, num_workers=6, generator=torch.Generator(device='cpu'),pin_memory=True, shuffle=True,persistent_workers=True, prefetch_factor=4)
    data_loader_val = DataLoader(validate_set, batch_size=128, num_workers=6, generator=torch.Generator(device='cpu'),pin_memory=True, shuffle=True,persistent_workers=True, prefetch_factor=4)
    data_loader_test = DataLoader(test_set, batch_size=64, num_workers=4, generator=torch.Generator(device='cpu'),persistent_workers=True)

    return data_loader, data_loader_val, data_loader_test





In [7]:
def model_train(lr, prepare_model, seed):
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True


    losses_tr = []
    accuracies_tr = []
    losses_val = []
    accuracies_val = []
    
    for lrate in lr:
        data_loader, data_loader_val, data_loader_test = prepare_transforms()
        print('LearningRate: '+ str(lrate))
    
        model, optimizer, criterion = prepare_model(lrate)
        
        train_losses = []
        train_accuracies = []
        val_losses = []
        val_accuracies = []
        
        num_epochs = 20
        
        prev_prev_loss = float('inf')
        prev_loss = float('inf')
        curr_loss = float('inf')
        
        for epoch in range(num_epochs): 
            print('Entered the loop')
            model.train()
            total_loss = 0
            correct = 0
            total = 0
            
            for i, (batch_X, batch_Y) in enumerate(data_loader):
                X = batch_X.to(device, non_blocking=True)  
                if(i == 1):
                    print('going')
                Y = batch_Y.to(device, non_blocking=True)  
                optimizer.zero_grad()
                outputs = model(X)
                loss = criterion(outputs, Y)
                loss.backward()
                optimizer.step()
                
                total_loss += loss.item()
                _, predicted = torch.max(outputs, 1)
                correct += (predicted == Y).sum().item()
                total += Y.size(0)
            

            avg_loss = total_loss / len(data_loader)
            train_accuracy = correct / total * 100
            train_losses.append(avg_loss)
            train_accuracies.append(train_accuracy)
            print(f"Epoch {epoch + 1}, Average Loss: {avg_loss:.4f}")
        
            model.eval()
            val_loss = 0
            correct = 0
            total = 0
            with torch.no_grad():
                for batch_X, batch_Y in data_loader_val:
                    X = batch_X.to(device, non_blocking=True)
                    Y = batch_Y.to(device, non_blocking=True)
                    outputs = model(X)
                    loss = criterion(outputs, Y)
                    val_loss += loss.item()
                    _, predicted = torch.max(outputs, 1)
                    correct += (predicted == Y).sum().item()
                    total += Y.size(0)
            avg_val_loss = val_loss / len(data_loader_val)
            val_accuracy = correct / total * 100
            print(f"Epoch {epoch + 1}, Validation Loss: {avg_val_loss:.4f}, Accuracy: {val_accuracy:.2f}%")
            
            val_losses.append(avg_val_loss)
            val_accuracies.append(val_accuracy)
    
    
            prev_prev_loss = prev_loss
            prev_loss = curr_loss
            curr_loss = avg_val_loss
        
            if(epoch == num_epochs - 1 ):
                losses_tr.append(train_losses)
                accuracies_tr.append(train_accuracies) 
                losses_val.append(val_losses)  
                accuracies_val.append(val_accuracies) 
    return losses_tr, accuracies_tr, losses_val, accuracies_val




In [8]:
results = []
lr = [0.001, 0.0005, 0.0001]
for i in range(3):
     results.append(model_train(lr, prepare_model_efficient_net, i))

LearningRate: 0.001


D:\anaconda\envs\torch_envi\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
D:\anaconda\envs\torch_envi\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Entered the loop
going
Epoch 1, Average Loss: 1.8964
Epoch 1, Validation Loss: 1.7583, Accuracy: 36.44%
Entered the loop
going
Epoch 2, Average Loss: 1.5984
Epoch 2, Validation Loss: 1.4898, Accuracy: 45.37%
Entered the loop
going
Epoch 3, Average Loss: 1.4447
Epoch 3, Validation Loss: 1.4325, Accuracy: 47.85%
Entered the loop
going
Epoch 4, Average Loss: 1.3392
Epoch 4, Validation Loss: 1.3124, Accuracy: 52.66%
Entered the loop
going
Epoch 5, Average Loss: 1.2503
Epoch 5, Validation Loss: 1.4698, Accuracy: 47.10%
Entered the loop
going
Epoch 6, Average Loss: 1.2847
Epoch 6, Validation Loss: 1.2562, Accuracy: 55.05%
Entered the loop
going
Epoch 7, Average Loss: 1.1661
Epoch 7, Validation Loss: 1.1980, Accuracy: 56.72%
Entered the loop
going
Epoch 8, Average Loss: 1.1227
Epoch 8, Validation Loss: 1.1613, Accuracy: 58.01%
Entered the loop
going
Epoch 9, Average Loss: 1.0766
Epoch 9, Validation Loss: 1.1644, Accuracy: 58.07%
Entered the loop
going
Epoch 10, Average Loss: 1.0803
Epoch 10, 

In [9]:
results

[([[1.896396836435253,
    1.5984010344201869,
    1.4446623218669132,
    1.3392044288868254,
    1.2502555649896914,
    1.2846737410873175,
    1.1660954192788764,
    1.1227205916392533,
    1.0765761206434532,
    1.0802768372337928,
    1.0993167562410235,
    1.0523471283641728,
    1.0453071816062385,
    0.9547779339111664,
    0.9503065279938958,
    0.9023698487911712,
    0.8814615334638141,
    0.9858805806460705,
    0.8746040218763731,
    0.8036879329857501],
   [1.9730042593384332,
    1.673728359863162,
    1.5184885078871793,
    1.428129217164083,
    1.3223173137415538,
    1.2527341944250194,
    1.1928628363053908,
    1.1457005945796317,
    1.0867845576087183,
    1.043328912505372,
    0.9994076538661664,
    0.9502070731568065,
    1.0934111700308593,
    0.9255639664320783,
    0.8577949001707814,
    0.8261074533008717,
    0.7823491135442798,
    0.7435439554974437,
    0.7271724405105818,
    0.6822288703088734],
   [2.1999576154418965,
    2.036997713487

([[1.896396836435253,
   1.5984010344201869,
   1.4446623218669132,
   1.3392044288868254,
   1.2502555649896914,
   1.2846737410873175,
   1.1660954192788764,
   1.1227205916392533,
   1.0765761206434532,
   1.0802768372337928,
   1.0993167562410235,
   1.0523471283641728,
   1.0453071816062385,
   0.9547779339111664,
   0.9503065279938958,
   0.9023698487911712,
   0.8814615334638141,
   0.9858805806460705,
   0.8746040218763731,
   0.8036879329857501],
  [1.9730042593384332,
   1.673728359863162,
   1.5184885078871793,
   1.428129217164083,
   1.3223173137415538,
   1.2527341944250194,
   1.1928628363053908,
   1.1457005945796317,
   1.0867845576087183,
   1.043328912505372,
   0.9994076538661664,
   0.9502070731568065,
   1.0934111700308593,
   0.9255639664320783,
   0.8577949001707814,
   0.8261074533008717,
   0.7823491135442798,
   0.7435439554974437,
   0.7271724405105818,
   0.6822288703088734],
  [2.1999576154418965,
   2.036997713487257,
   1.9058921797709032,
   1.805483071